In [1]:
from selenium import webdriver
from time import sleep

driver = webdriver.Chrome('/Users/iliasmiraoui/Desktop/chromedriver')

In [238]:

artist= "Coldplay"
url = "https://www.riaa.com/gold-platinum/?tab_active=default-award&ar="+artist+"&lab=&genre=&format=&date_option=release&from=&to=&award=&type=&category=&adv=SEARCH#search_section"

In [283]:
driver.get(url)

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=51504): Max retries exceeded with url: /session/38bbee87c37962ffa2f907a75b41a839/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x110fcae90>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [245]:
artists = []
songs = []
last_certification = []
label = []
frmat= []
details = []
nb_songs = len(driver.find_elements_by_xpath("//*[@class='table_award_row']"))
if len(driver.find_element_by_xpath("//*[@id='loadmore']"))>0:
    driver.find_element_by_xpath("//*[@id='loadmore']").click()
to_click = driver.find_elements_by_xpath("//*[@class='others_cell format_cell']/div/a")
for link in to_click:
    link.click()
    #sleep(0.3)
awards = driver.find_elements_by_xpath("//*[@class='table_award_row expanded']")
for award in awards:
    artists.append(award.find_element_by_xpath(".//*[@class='artists_cell']").text)
    songs.append(award.find_element_by_xpath(".//*[@class='others_cell']").text)
    last_certification.append(award.find_elements_by_xpath(".//*[@class='others_cell']")[1].text)
    label.append(award.find_elements_by_xpath(".//*[@class='others_cell']")[2].text)
    frmat.append(award.find_element_by_xpath(".//*[@class='others_cell format_cell']").text)
    next_element = award.find_element_by_xpath(".//following-sibling::tr")
    detail = next_element.find_element_by_xpath("//*[@class='table_award_row expanded']")
    details.append([x.text for x in next_element.find_elements_by_xpath(".//*[@class='content_recent_table']/td")])

['SOMETHING JUST LIKE THIS', 'A SKY FULL OF STARS', 'HYMN FOR THE WEEKEND', 'ADVENTURE OF A LIFETIME', 'A HEAD FULL OF DREAMS', 'GHOST STORIES', 'MAGIC', 'PARADISE', 'MYLO XYLOTO', 'VIVA LA VIDA', 'VIVA LA VIDA', 'FIX YOU', 'X&Y', 'A RUSH OF BLOOD TO THE HEAD', 'YELLOW', 'THE SCIENTIST', 'SPEED OF SOUND', 'CLOCKS', 'COLDPLAY LIVE 2003', 'PARACHUTES', 'COLDPLAY LIVE 2003']
['SINGLE\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'ALBUM\nMORE DETAILS', 'ALBUM\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'ALBUM\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'ALBUM\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'ALBUM\nMORE DETAILS', 'ALBUM\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'SINGLE\nMORE DETAILS', 'VIDEO LONGFORM\nMORE DETAILS', 'ALBUM\nMORE DETAILS', 'ALBUM\nMORE DETAILS']
[['FEBRUARY 17, 2017', '6x Multi-Platinum | November 8, 2019', 'None', 'Digital', '6 Million', 'POP', ' ', '5x Multi-Pla

In [6]:
def get_riaa(artist):
    query = artist
    query = urllib.parse.quote_plus(query)
    df = pd.DataFrame(columns=["artist","artists","songs","last_certification","label","format","details"])
    url = "https://www.riaa.com/gold-platinum/?tab_active=default-award&ar="+query+"&lab=&genre=&format=&date_option=release&from=&to=&award=&type=&category=&adv=SEARCH#search_section"
    driver.get(url)
    sleep(0.1)
    artists = []
    songs = []
    last_certification = []
    label = []
    frmat= []
    details = []
    hascontent =0
    try:
        driver.find_element_by_xpath("//*[@id='loadmore']").click()
    except:
        hascontent =1
        sleep(0.3)
    to_click = driver.find_elements_by_xpath("//*[@class='others_cell format_cell']/div/a")
    for link in to_click:
        link.click()
        sleep(0.6)
    awards = driver.find_elements_by_xpath("//*[@class='table_award_row expanded']")
    if hascontent ==1:
        awards.extend(driver.find_elements_by_xpath("//*[@class='table_award_row expanded hasContent']"))
    nb_songs = len(awards)
    for award in awards:
        artists.append(award.find_element_by_xpath(".//*[@class='artists_cell']").text)
        songs.append(award.find_element_by_xpath(".//*[@class='others_cell']").text)
        last_certification.append(award.find_elements_by_xpath(".//*[@class='others_cell']")[1].text)
        label.append(award.find_elements_by_xpath(".//*[@class='others_cell']")[2].text)
        frmat.append(award.find_element_by_xpath(".//*[@class='others_cell format_cell']").text)
        next_element = award.find_element_by_xpath(".//following-sibling::tr")
        detail = next_element.find_element_by_xpath("//*[@class='table_award_row expanded']")
        details.append([x.text for x in next_element.find_elements_by_xpath(".//*[@class='content_recent_table']/td")])
    for i in range(nb_songs):
        df.loc[i] = [artist,artists[i],songs[i],last_certification[i],label[i],frmat[i],details[i]]
    return df

In [8]:
import pandas as pd
import urllib.parse
import pickle 
with open('artist_df.pkl', 'rb') as f:
        artist_df = pickle.load(f)
with open('song_df_100.pkl', 'rb') as f:
        song_df = pickle.load(f)


In [17]:
from tqdm import tqdm_notebook as tqdm
driver = webdriver.Chrome('/Users/iliasmiraoui/Desktop/chromedriver')
pbar = tqdm(song_df["artist"].unique())
i = 0
for artist in pbar:
    if (artist not in artist_df["artist"].unique()) and (artist not in artist_df[0].unique()):
        i=i+1
        df = get_riaa(artist)
        artist_df = artist_df.append(df)
        if len(df) == 0:
            artist_df = artist_df.append([artist,None,None,None,None,None,None])
        print(i)
        if i % 5 == 0:
            with open('artist_df.pkl', 'wb') as f:
                pickle.dump(artist_df, f)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258



In [18]:
with open('artist_df.pkl', 'wb') as f:
    pickle.dump(artist_df, f)

In [20]:
import numpy as np
keywords = ["Featuring","Feauring", "featuring", "&",","," X ", " x ","Presents", "From", "Or","And",]
new_df = artist_df.dropna(subset=[0])
temp_list = np.asarray(new_df[0])
count=0
for keyword in keywords:
    artist_list = [i.split(keyword) for i in temp_list]
    temp_list = list()
    count=count+1
    for i in artist_list:
            temp_list.extend(i)

artist_list = list()
for i in temp_list:
    i = i.strip()
    if len(i)>1:
        artist_list.append(i)
artist_list = set(artist_list)        
artist_list_new = set(artist_list) 

for i in artist_list:
    if len(artist_df[artist_df[0] == i]) >0 and i in artist_list_new:
        artist_list_new.remove(i)
    elif len(artist_df[artist_df["artist"] == i]) >0 and i in artist_list_new:
        artist_list_new.remove(i)
    

In [76]:
from tqdm import tqdm_notebook as tqdm
driver = webdriver.Chrome('/Users/iliasmiraoui/Desktop/chromedriver')
pbar = list(artist_list_new)
i = 0
for artist in pbar:
    if (artist not in artist_df["artist"].unique()) and (artist not in artist_df[0].unique()):
        i=i+1
        df = get_riaa(artist)
        artist_df = artist_df.append(df)
        if len(df) == 0:
            artist_df = artist_df.append([artist,None,None,None,None,None,None])
        print(i)
        if i % 5 == 0:
            with open('artist_df.pkl', 'wb') as f:
                pickle.dump(artist_df, f)
with open('artist_df.pkl', 'wb') as f:
    pickle.dump(artist_df, f)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61


In [28]:
artist_df = artist_df.drop([0],axis=1)  #delete first column, no information
artist_df = artist_df.replace(to_replace='None', value=np.nan).dropna() 

In [79]:
artist_df = artist_df.reset_index()
artist_df.pop("index")
artist_df

,artist,artists,details,format,label,last_certification,songs,0
0,Lewis Capaldi,LEWIS CAPALDI,"[NOVEMBER 8, 2018, Platinum | August 15, 2019,...",SINGLE\nMORE DETAILS,CAPITOL RECORDS,"August 15, 2019",SOMEONE YOU LOVED,NaN
1,Post Malone,POST MALONE & SWAE LEE,"[OCTOBER 18, 2018, 8x Multi-Platinum | October...",SINGLE\nMORE DETAILS,REPUBLIC RECORDS,"October 22, 2019",SUNFLOWER (SPIDER-MAN: INTO THE SPIDER-VERSE),NaN
2,Post Malone,POST MALONE,"[DECEMBER 9, 2016, 10x Multi-Platinum | Octobe...",SINGLE\nMORE DETAILS,REPUBLIC RECORDS,"October 8, 2019",CONGRATULATIONS,NaN
3,Post Malone,POST MALONE,"[APRIL 27, 2018, Platinum | March 13, 2019, No...",SINGLE\nMORE DETAILS,REPUBLIC RECORDS,"March 13, 2019",BALL FOR ME,NaN
4,Post Malone,POST MALONE,"[APRIL 27, 2018, 3x Multi-Platinum | March 13,...",ALBUM\nMORE DETAILS,REPUBLIC RECORDS,"March 13, 2019",BEERBONGS & BENTLEYS,NaN
...,...,...,...,...,...,...,...,...
13368,Sons,ALAN PARSONS PROJECT,"[NO RELEASE DATE AVAILABLE., Gold | July 24, 1...",ALBUM\nMORE DETAILS,ARISTA,"July 24, 1978",PYRAMID,NaN
13369,Sons,THE JACKSONS,"[NO RELEASE DATE AVAILABLE., Gold | April 5, 1...",ALBUM\nMORE DETAILS,EPIC,"April 5, 1977",THE JACKSONS,NaN
13370,Sons,THE FOUR SEASONS,"[NO RELEASE DATE AVAILABLE., Gold | March 29, ...",SINGLE\nMORE DETAILS,CURB,"March 29, 1976","DECEMBER, 1963 (OH, WHAT A NIGHT)",NaN
13371,Sons,THE FOUR SEASONS,"[NO RELEASE DATE AVAILABLE., Gold | October 8,...",ALBUM\nMORE DETAILS,PHILLIPS,"October 8, 1970",EDIZIONE D'ORO,NaN


In [103]:
from fuzzywuzzy import fuzz

def similar(a, b, threshold=101):
    return (fuzz.partial_ratio(a, b) >= threshold)

for row in artist_df.index:
    if not similar(artist_df.loc[row,"artist"].upper(),artist_df.loc[row,"artists"]) and (similar(artist_df.loc[row,"artist"].upper(),artist_df.loc[row,"artists"],95)) and (fuzz.ratio(artist_df.loc[row,"artist"].upper(),artist_df.loc[row,"artists"]) <=95):
        print(fuzz.partial_ratio(artist_df.loc[row,"artist"].upper(),artist_df.loc[row,"artists"]))
        print(row, artist_df.loc[row,"artist"],artist_df.loc[row,"artists"])   


100
1 Post Malone POST MALONE & SWAE LEE
100
22 Shawn Mendes & Camila Cabello SHAWN MENDES
100
23 Shawn Mendes & Camila Cabello SHAWN MENDES
100
24 Shawn Mendes & Camila Cabello SHAWN MENDES
100
25 Shawn Mendes & Camila Cabello SHAWN MENDES
100
27 Shawn Mendes & Camila Cabello SHAWN MENDES
100
28 Shawn Mendes & Camila Cabello SHAWN MENDES
100
29 Shawn Mendes & Camila Cabello SHAWN MENDES
100
30 Shawn Mendes & Camila Cabello SHAWN MENDES
100
31 Shawn Mendes & Camila Cabello SHAWN MENDES
100
32 Shawn Mendes & Camila Cabello SHAWN MENDES
100
33 Shawn Mendes & Camila Cabello SHAWN MENDES
100
35 Shawn Mendes & Camila Cabello SHAWN MENDES
100
36 Shawn Mendes & Camila Cabello SHAWN MENDES
100
37 Shawn Mendes & Camila Cabello SHAWN MENDES
100
38 Shawn Mendes & Camila Cabello SHAWN MENDES
100
39 Shawn Mendes & Camila Cabello SHAWN MENDES
100
40 Shawn Mendes & Camila Cabello SHAWN MENDES
100
46 Selena Gomez KYGO & SELENA GOMEZ
100
47 Selena Gomez SELENA GOMEZ X MARSHMELLO
100
49 Selena Gomez DJ 

100
1110 5 Seconds Of Summer THE CHAINSMOKERS & 5 SECONDS OF SUMMER
100
1133 Ariana Grande ARIANA GRANDE & JOHN LEGEND
100
1134 Ariana Grande JESSIE J, ARIANA GRANDE, NICKI MINAJ
100
1201 The Chainsmokers & Bebe Rexha THE CHAINSMOKERS
100
1202 The Chainsmokers & Bebe Rexha THE CHAINSMOKERS
100
1204 The Chainsmokers & Bebe Rexha THE CHAINSMOKERS
100
1207 The Chainsmokers & Bebe Rexha THE CHAINSMOKERS
100
1208 The Chainsmokers & Bebe Rexha THE CHAINSMOKERS
100
1209 The Chainsmokers & Bebe Rexha THE CHAINSMOKERS
100
1210 The Chainsmokers & Bebe Rexha THE CHAINSMOKERS
100
1211 The Chainsmokers & Bebe Rexha THE CHAINSMOKERS
100
1213 The Chainsmokers & Bebe Rexha THE CHAINSMOKERS
100
1214 The Chainsmokers & Bebe Rexha THE CHAINSMOKERS
100
1215 The Chainsmokers & Bebe Rexha THE CHAINSMOKERS
100
1216 The Chainsmokers & Bebe Rexha THE CHAINSMOKERS
100
1217 The Chainsmokers & Bebe Rexha THE CHAINSMOKERS
100
1218 The Chainsmokers & Bebe Rexha THE CHAINSMOKERS
100
1219 The Chainsmokers & Bebe Rexh

1833 YG TYGA
100
1834 YG YG F/TYGA & JON Z
100
1835 YG TYGA
100
1836 YG KYGO & JUSTIN JESSO
100
1837 YG TYGA
100
1838 YG KID INK, TYGA, WALE, YG & RICH HOMIE QUAN
100
1842 YG TYGA
100
1844 YG KYGO & ELLIE GOULDING
100
1846 YG KYGO
100
1848 YG CHRIS BROWN X TYGA
100
1849 YG KYGO FEAT. CONRAD SEWELL
100
1850 YG KYGO FEAT. PARSON JAMES
100
1851 YG KYGO FEAT. MATY NOYES
100
1855 YG YO GOTTI FEAT. JEEZY & YG
100
1856 YG TYGA
100
1857 YG TYGA
100
1858 YG TYGA
100
1859 YG TYGA
100
1860 YG TYGA
100
1865 Imagine Dragons LIL WAYNE, WIZ KHALIFA & IMAGINE DRAGONS WITH LOGIC AND TY DOLLA $IGN (FEAT. X AMBASSADORS)
100
1924 Logic LIL WAYNE, WIZ KHALIFA & IMAGINE DRAGONS WITH LOGIC AND TY DOLLA $IGN (FEAT. X AMBASSADORS)
100
1954 Chris Brown CHRIS BROWN & T-PAIN
100
1989 Kodak Black PNB ROCK, KODAK BLACK & A BOOGIE WIT DA HOODIE
100
1995 Bebe Rexha MACHINE GUN KELLY, X AMBASSADORS & BEBE REXHA
100
1999 Bebe Rexha MARTIN GARRIX & BEBE REXHA
100
2000 Bebe Rexha G-EAZY X BEBE REXHA
100
2002 Iggy Azalea 

100
3548 Brad Paisley BRAD PAISLEY & CARRIE UNDERWOOD
100
3565 PnB Rock YOUNG THUG, 2 CHAINZ, WIZ KHALIFA & PNB ROCK
100
3566 PnB Rock PNB ROCK, KODAK BLACK & A BOOGIE WIT DA HOODIE
100
3572 John Legend ARIANA GRANDE & JOHN LEGEND
100
3590 Sia ALESSIA CARA
100
3591 Sia DESCEMER BUENO & ENRIQUE IGLESIAS
100
3592 Sia RVSSIAN
100
3593 Sia RVSSIAN
100
3594 Sia ENRIQUE IGLESIAS
100
3596 Sia ALESSIA CARA
100
3597 Sia ALESSIA CARA
100
3599 Sia ALESSIA CARA
100
3600 Sia ENRIQUE IGLESIAS
100
3601 Sia ENRIQUE IGLESIAS
100
3604 Sia ALESSIA CARA
100
3607 Sia ENRIQUE IGLESIAS
100
3608 Sia GABRIEL IGLESIAS
100
3609 Sia ENRIQUE IGLESIAS
100
3610 Sia GABRIEL IGLESIAS
100
3611 Sia GABRIEL IGLESIAS
100
3612 Sia ENRIQUE IGLESIAS
100
3613 Sia ENRIQUE IGLESIAS
100
3614 Sia ENRIQUE IGLESIAS
100
3615 Sia ENRIQUE IGLESIAS
100
3616 Sia GABRIEL IGLESIAS
100
3617 Sia ENRIQUE IGLESIAS
100
3618 Sia FANTASIA
100
3619 Sia FANTASIA
100
3642 X Ambassadors X AMBASSADORS & JAMIE N COMMONS
100
3643 X Ambassadors MACHINE 

100
6001 Reba REBA MC ENTIRE
100
6002 Reba REBA MC ENTIRE
100
6003 Reba REBA MC ENTIRE
100
6004 Reba REBA MC ENTIRE
100
6005 Reba REBA MC ENTIRE
100
6016 R. Kelly JAY-Z & R. KELLY
100
6022 R. Kelly JAY-Z & R. KELLY
100
6027 R. Kelly R. KELLY & CELINE DION
100
6031 R. Kelly R. KELLY FEAT. R. ISLEY
100
6036 R. Kelly R. KELLY & PUBLIC ANNOUNCEMENT
100
6049 Monica MONICA NARANJO
100
6050 Monica MONICA NARANJO
100
6054 Monica BRANDY & MONICA
100
6067 Fantasia TROPICAL FANTASIA
100
6073 Sean Kingston SEAN KINGSTON & JUSTIN BIEBER
100
6092 Rob Thomas SANTANA FEAT. ROB THOMAS
100
6095 Alicia Keys JAY-Z & ALICIA KEYS
100
6102 Alicia Keys ALICIA KEYS & USHER
100
6107 Alicia Keys ALICIA KEYS & USHER
100
6121 Hannah Montana MILEY CYRUS AS HANNAH MONTANA
100
6122 Hannah Montana MILEY CYRUS AS HANNAH MONTANA
100
6132 Jaheim NELLY AND JAHEIM
100
6207 T-Pain CHRIS BROWN & T-PAIN
100
6208 T-Pain E-40 FEAT. T-PAIN & KANDI
100
6209 T-Pain T-PAIN FEAT. B.O.B.
100
6210 T-Pain T-PAIN FEAT. LUDACRIS
100
6211

...

11539 bit XZIBIT
100
11540 bit EDDIE RABBITT
100
11541 bit EDDIE RABBITT
100
11542 bit EDDIE RABBITT
100
11543 bit EDDIE RABBITT
100
11544 bit EDDIE RABBITT
100
11547 Jeannie JEANNIE C. RILEY
100
11548 Jeannie JEANNIE C. RILEY
100
11580 Thalia THALIA & LALI
100
11582 Thalia THALIA & NATTI NATASHA
100
11593 Ja JANIS JOPLIN
100
11594 Ja RAUW ALEJANDRO X NICKY JAM
100
11595 Ja LIL TJAY
100
11596 Ja NOTD & FELIX JAEHN FT. GEORGIA KU & CAPTAIN CUTS
100
11597 Ja RAUW ALEJANDRO X CHENCHO CORLEONE
100
11598 Ja X AMBASSADORS & JAMIE N COMMONS
100
11599 Ja LIL TJAY
100
11600 Ja GUAYNAA, NICKY JAM & FARRUKO
100
11601 Ja JAY ROCK
100
11602 Ja AMBJAAY
100
11603 Ja NORIEL, YANDEL, & NICKY JAM
100
11604 Ja BREAKING BENJAMIN
100
11605 Ja JAMES BLUNT
100
11606 Ja AULI'I CRAVALHO, CHRISTOPHER JACKSON, LOUISE BUSH, NICOLE SCHERZINGER, RACHEL HOUSE
100
11607 Ja COSCULLUELA & NICKY JAM
100
11608 Ja NIO GARCIA, CASPER MAGICO, BAD BUNNY, OZUNA, DARELL & NICKY JAM
100
11609 Ja JASON ALDEAN
100
11610 Ja NIO GA

100
12069 MO THE CHAINSMOKERS
100
12070 MO MARINA & THE DIAMONDS
100
12071 MO PARAMORE
100
12072 MO PARAMORE
100
12073 MO PARAMORE
100
12074 MO PARAMORE
100
12075 MO MARINA & THE DIAMONDS
100
12076 MO LIL MOSEY
100
12077 MO SMOKEPURPP
100
12078 MO THE CHAINSMOKERS
100
12081 Alvin J BALVIN & BAD BUNNY
100
12082 Alvin CALVIN HARRIS & SAM SMITH
100
12083 Alvin BENNY BLANCO, SELENA GOMEZ, J BALVIN, TAINY
100
12084 Alvin BAD BUNNY, PRINCE ROYCE & J BALVIN, DJ LUIAN & MAMBO KINGZ)
100
12085 Alvin J BALVIN & SEAN PAUL
100
12086 Alvin J BALVIN & BAD BUNNY
100
12087 Alvin J BALVIN & BAD BUNNY
100
12088 Alvin J BALVIN & BAD BUNNY
100
12089 Alvin J BALVIN & BAD BUNNY
100
12090 Alvin J BALVIN & BAD BUNNY
100
12091 Alvin J BALVIN & BAD BUNNY (FEAT MARCIANO CANTERO)
100
12092 Alvin J BALVIN & BAD BUNNY
100
12093 Alvin CALVIN HARRIS & RIHANNA
100
12094 Alvin SZA X CALVIN HARRIS
100
12095 Alvin CALVIN HARRIS, DUA LIPA
100
12096 Alvin NICKY JAM & J BALVIN
100
12097 Alvin JHAY CORTEZ & J BALVIN
100
1209

100
12464 Lea CREEDENCE CLEARWATER REVIVAL
100
12465 Lea CREEDENCE CLEARWATER REVIVAL
100
12466 Lea CLEAN BANDIT
100
12467 Lea CREEDENCE CLEARWATER REVIVAL
100
12468 Lea CLEAN BANDIT
100
12469 Lea CREEDENCE CLEARWATER REVIVAL
100
12470 Lea CREEDENCE CLEARWATER REVIVAL
100
12471 Lea CREEDENCE CLEARWATER REVISITED
100
12472 Lea CLEAN BANDIT
100
12473 Lea CLEAN BANDIT
100
12474 Lea LEA SALONGA & BRAD KANE
100
12475 Lea IGGY AZALEA
100
12476 Lea IGGY AZALEA
100
12477 Lea CREEDENCE CLEARWATER REVIVAL
100
12478 Lea CREEDENCE CLEARWATER REVIVAL
100
12479 Lea CREEDENCE CLEARWATER REVIVAL
100
12480 Lea IGGY AZALEA
100
12481 Lea IGGY AZALEA
100
12482 Lea BLEACHERS
100
12483 Lea IGGY AZALEA
100
12484 Lea T.I. FEAT. IGGY AZALEA
100
12485 Lea IGGY AZALEA
100
12486 P. Diddy NELLY, P. DIDDY & MURPHY LEE
100
12487 P. Diddy P. DIDDY & THE FAMILY
100
12490 P. Diddy P. DIDDY FEAT. JIMMY PAGE
100
12491 P. Diddy P. DIDDY & THE FAMILY
100
12492 P. Diddy P. DIDDY & THE FAMILY
100
12493 P. Diddy P. DIDDY & FA

100
12879 Sech SECH, JUSTIN QUILES, DE LA GHETTO & DIMELO FLOW
100
12881 Dolla TY DOLLA $IGN
100
12882 Dolla JUICY J, WIZ KHALIFA, & TY DOLLA $IGN
100
12883 Dolla TY DOLLA $IGN
100
12884 Dolla TY DOLLA $IGN
100
12885 Dolla TY DOLLA $IGN
100
12886 Dolla TY DOLLA $IGN
100
12887 Dolla TY DOLLA $IGN
100
12888 Dolla LIL WAYNE, WIZ KHALIFA & IMAGINE DRAGONS WITH LOGIC AND TY DOLLA $IGN (FEAT. X AMBASSADORS)
100
12889 Dolla TY DOLLA $IGN
100
12890 Dolla TY DOLLA $IGN
100
12891 Dolla TY DOLLA $IGN
100
12892 Hollis DAVE HOLLISTER
100
12893 Hollis DAVE HOLLISTER
100
12895 Natti Natasha SEBASTIAN YATRA, DADDY YANKEE & NATTI NATASHA (FEAT JONAS BROTHERS)
100
12896 Natti Natasha SILVESTRE DANGOND & NATTI NATASHA
100
12897 Natti Natasha THALIA & NATTI NATASHA
100
12898 Natti Natasha BECKY G, NATTI NATASHA
100
12899 Natti Natasha NATTI NATASHA, OZUNA
100
12900 Natti Natasha NATTI NATASHA & BAD BUNNY
100
12901 Big Pun BIG PUNISHER
100
12902 Big Pun BIG PUNISHER
100
12906 Illenium GRYFFIN & ILLENIUM
10

100
13208 Big A GREAT BIG WORLD
100
13209 Big BIG PUNISHER
100
13210 Big BIG SEAN
100
13211 Big BIG SEAN
100
13212 Big BIG SEAN
100
13213 Big LITTLE BIG TOWN
100
13214 Big BIG SEAN
100
13215 Big BIG SEAN
100
13216 Big BIG SEAN
100
13217 Big BIG SEAN
100
13218 Big A GREAT BIG WORLD & CHRISTINA AGUILERA
100
13219 Big BIG & RICH
100
13220 Big LITTLE BIG TOWN
100
13221 Big BIG & RICH
100
13222 Big BIG & RICH
100
13223 Big LITTLE BIG TOWN
100
13224 Big BIG SEAN
100
13225 Big BIG SEAN
100
13226 Big BIG DATA
100
13227 Forever JACOB FOREVER
100
13228 Forever LARY OVER, EL MICHA, JACOB FOREVER
100
13229 Forever JACOB FOREVER
100
13230 Forever JACOB FOREVER
100
13231 Forever ARTISTS OF THEN, NOW & FOREVER
100
13232 AlunaGeorge DJ SNAKE & ALUNAGEORGE
100
13233 Jessica JESSICA SIMPSON
100
13234 Jessica JESSICA SIMPSON
100
13235 Jessica JESSICA SIMPSON
100
13236 Jessica JESSICA SIMPSON
100
13237 Jessica JESSICA SIMPSON
100
13238 Jessica JESSICA SIMPSON
100
13239 Jessica JESSICA SIMPSON
100
13240 Je

In [105]:
to_drop = [346,354,360,1831,1832,1833,1835,1842,1844,1846,1849,1850,1851,1855,1856,2441,2443,2447,229,2450,2452,
          2849,3278,3282,3286,3296,3297,3298,3299,3300,3301,3302,3523,3590,3604,3765,3769,3809,3810,3811,4057,4079,
           4083,4086,4194,4198,4199,4209,4213,4216,4303,4404,4458,4534,5260,5262,5572,5793,5794,5795,5916,5918,5919,
           5920,5922,6049,6050,6067,6240,6253,6254,6255,6276,6279,6278,6280,6283,6287,6286,6289,6290,6291,6293,6306,
           6815,7053,7056,7271,7272,7273,7274,7597,7598,7599,7600,7791,7793,7794,7795,7798,7799,7800,7804,7805,7812,7818,
           8033,8035,8036,8037,8092,8142,8145,8147,8259,8284,8610,8611,8666,8670,8671,8672,8675,8676,8678,8683,8753,8785,8786,8787,
           8799,8800,8809,8903,8908,8909,8912,8926,8927,9231,9233,9289,9290,9310,9315,9394,9396,9512,9567,9568,9569,9571,
           9602,9627,9628,9771,9796,9799,9800,9846,9848,9851,9891,9892,9893,9894,9957,9959,10043,10044,10169,10170,10204,
           10205,10206,10328,10329,10505,10506,10529,10567,10573,10624,10695,10745,10746,10915,10989,10995,11100,11133,
           11141,11144,11234,11297,11298,11341,11348,11520,11521,11522,11624,11625,11626,11807,11808,11809,11814,11817,
           12298,12495,12501,12502,12503,12931,12936,12937,
          ]

to_drop.extend([i for i in range(13342,13372)])
to_drop.extend([i for i in range(13268,13309)])
to_drop.extend([i for i in range(13233,13259)])
to_drop.extend([i for i in range(13191,13232)])
to_drop.extend([i for i in range(13159,13187)])
to_drop.extend([i for i in range(13147,13158)])
to_drop.extend([i for i in range(13105,13135)])
to_drop.extend([i for i in range(13061,13086)])
to_drop.extend([i for i in range(13013,13043)])
to_drop.extend([i for i in range(12951,12999)])
to_drop.extend([i for i in range(12907,12930)])
to_drop.extend([i for i in range(12895,12903)])
to_drop.extend([i for i in range(12881,12894)])
to_drop.extend([i for i in range(12844,12855)])
to_drop.extend([i for i in range(12813,12843)])
to_drop.extend([i for i in range(12758,12794)])
to_drop.extend([i for i in range(12743,12756)])
to_drop.extend([i for i in range(12694,12740)])
to_drop.extend([i for i in range(12643,12672)])
to_drop.extend([i for i in range(12592,12626)])
to_drop.extend([i for i in range(12547,12577)])
to_drop.extend([i for i in range(12458,12486)])
to_drop.extend([i for i in range(12391,12420)])
to_drop.extend([i for i in range(12374,12377)])
to_drop.extend([i for i in range(12303,12364)])
to_drop.extend([i for i in range(12239,12269)])
to_drop.extend([i for i in range(12196,12226)])
to_drop.extend([i for i in range(12146,12176)])
to_drop.extend([i for i in range(12081,12111)])
to_drop.extend([i for i in range(12027,12079)])
to_drop.extend([i for i in range(12017,12025)])
to_drop.extend([i for i in range(11986,11997)])
to_drop.extend([i for i in range(11945,11980)])
to_drop.extend([i for i in range(11904,11935)])
to_drop.extend([i for i in range(11868,11887)])
to_drop.extend([i for i in range(11831,11860)])
to_drop.extend([i for i in range(11818,11827)])
to_drop.extend([i for i in range(11810,11813)])
to_drop.extend([i for i in range(11776,11798)])
to_drop.extend([i for i in range(11773,11775)])
to_drop.extend([i for i in range(11593,11623)])
to_drop.extend([i for i in range(11547,11549)])
to_drop.extend([i for i in range(11537,11545)])
to_drop.extend([i for i in range(11527,11534)])
to_drop.extend([i for i in range(11484,11503)])
to_drop.extend([i for i in range(11407,11467)])
to_drop.extend([i for i in range(11357,11394)])
to_drop.extend([i for i in range(11235,11251)])
to_drop.extend([i for i in range(11147,11184)])
to_drop.extend([i for i in range(11137,11140)])
to_drop.extend([i for i in range(11102,11129)])
to_drop.extend([i for i in range(11086,11098)])
to_drop.extend([i for i in range(11065,11083)])
to_drop.extend([i for i in range(11000,11011)])
to_drop.extend([i for i in range(10997,10999)])
to_drop.extend([i for i in range(10917,10931)])
to_drop.extend([i for i in range(10877,10882)])
to_drop.extend([i for i in range(10822,10829)])
to_drop.extend([i for i in range(10809,10815)])
to_drop.extend([i for i in range(10767,10796)])
to_drop.extend([i for i in range(10749,10758)])
to_drop.extend([i for i in range(10709,10743)])
to_drop.extend([i for i in range(10699,10707)])
to_drop.extend([i for i in range(10658,10686)])
to_drop.extend([i for i in range(10625,10639)])
to_drop.extend([i for i in range(10593,10622)])
to_drop.extend([i for i in range(10475,10496)])
to_drop.extend([i for i in range(10445,10451)])
to_drop.extend([i for i in range(10331,10340)])
to_drop.extend([i for i in range(10296,10324)])
to_drop.extend([i for i in range(10269,10277)])
to_drop.extend([i for i in range(10237,10267)])
to_drop.extend([i for i in range(10109,10139)])
to_drop.extend([i for i in range(10079,10108)])
to_drop.extend([i for i in range(10045,10072)])
to_drop.extend([i for i in range(10010,10040)])
to_drop.extend([i for i in range(9984,9995)])
to_drop.extend([i for i in range(9940,9944)])
to_drop.extend([i for i in range(9913,9935)])
to_drop.extend([i for i in range(9852,9882)])
to_drop.extend([i for i in range(9712,9740)])
to_drop.extend([i for i in range(9539,9562)])
to_drop.extend([i for i in range(9438,9468)])
to_drop.extend([i for i in range(9316,9346)])
to_drop.extend([i for i in range(9291,9300)])
to_drop.extend([i for i in range(9250,9280)])
to_drop.extend([i for i in range(9197,9201)])
to_drop.extend([i for i in range(9076,9080)])
to_drop.extend([i for i in range(9006,9011)])
to_drop.extend([i for i in range(8877,8892)])
to_drop.extend([i for i in range(362,376)])
to_drop.extend([i for i in range(1836,1830)])
to_drop.extend([i for i in range(1857,1861)])
to_drop.extend([i for i in range(2427,2430)])
to_drop.extend([i for i in range(2427,2430)])
to_drop.extend([i for i in range(2803,2808)])
to_drop.extend([i for i in range(2427,2430)])
to_drop.extend([i for i in range(2962,2967)])
to_drop.extend([i for i in range(3481,3493)])
to_drop.extend([i for i in range(3591,3602)])
to_drop.extend([i for i in range(3607,3620)])
to_drop.extend([i for i in range(3812,3838)])
to_drop.extend([i for i in range(4058,4073)])
to_drop.extend([i for i in range(4268,4276)])
to_drop.extend([i for i in range(4277,4297)])
to_drop.extend([i for i in range(4303,4332)])
to_drop.extend([i for i in range(4410,4425)])
to_drop.extend([i for i in range(5064,5069)])
to_drop.extend([i for i in range(5574,5589)])
to_drop.extend([i for i in range(5873,5903)])
to_drop.extend([i for i in range(5923,5931)])
to_drop.extend([i for i in range(6296,6305)])
to_drop.extend([i for i in range(6310,6322)])
to_drop.extend([i for i in range(6787,6791)])
to_drop.extend([i for i in range(6822,6827)])
to_drop.extend([i for i in range(6832,6862)])
to_drop.extend([i for i in range(7009,7039)])
to_drop.extend([i for i in range(7057,7065)])
to_drop.extend([i for i in range(7066,7096)])
to_drop.extend([i for i in range(7136,7165)])
to_drop.extend([i for i in range(7257,7270)])
to_drop.extend([i for i in range(7779,7790)])
to_drop.extend([i for i in range(7808,7812)])
to_drop.extend([i for i in range(7822,7831)])
to_drop.extend([i for i in range(7840,7843)])
to_drop.extend([i for i in range(7846,7863)])
to_drop.extend([i for i in range(7871,7873)])
to_drop.extend([i for i in range(7876,7899)])
to_drop.extend([i for i in range(7257,7270)])
to_drop.extend([i for i in range(8148,8153)])
to_drop.extend([i for i in range(8264,8274)])
to_drop.extend([i for i in range(8396,8411)])
to_drop.extend([i for i in range(8418,8437)])
to_drop.extend([i for i in range(8438,8441)])
to_drop.extend([i for i in range(8442,8447)])
to_drop.extend([i for i in range(8495,8503)])
to_drop.extend([i for i in range(8506,8523)])
to_drop.extend([i for i in range(8653,8665)])
to_drop.extend([i for i in range(8792,8797)])
to_drop.extend([i for i in range(8815,8840)])

for i in to_drop:
    if i in artist_df.index:
        artist_df =artist_df.drop(i)

        
#REBA?        
        
        
# Remove "John" or absurd numbers
# Productions # Crews #Tony Gqrcia & Dawn #Men #Diddy
#Melle #Foxxe #STING #Mo #Luke #Big T #Bun B #Die #Tone #Houston #Young D  #Rita
#erson #Evan #Mez #Mary Mary #Big #Rich #Jessica #Teo 

# CASH CASH is off #Bas is off #RBE #Keith

In [92]:
to_drop = [244,3772,4554,4726,6124,6126,6127,7391,7591,8000,8001,8002,8649,8910,8911,8928,10876,11101,11349,11350,11353,
          11507,11508,11509,11656,12283,12425]

to_drop.extend([i for i in range(4649,4653)])
to_drop.extend([i for i in range(4730,4736)])
to_drop.extend([i for i in range(5843,5873)])
to_drop.extend([i for i in range(7990,7999)])
to_drop.extend([i for i in range(8904,8908)])
to_drop.extend([i for i in range(8913,8917)])
to_drop.extend([i for i in range(9067,9069)])
to_drop.extend([i for i in range(10370,10386)])
to_drop.extend([i for i in range(11026,11056)])
to_drop.extend([i for i in range(11658,11686)])
to_drop.extend([i for i in range(11827,11830)])
to_drop.extend([i for i in range(11892,11901)])

for i in to_drop:
    if i in artist_df.index:
        artist_df =artist_df.drop(i)
# Remove "John" or absurd numbers
# Productions # Crews #Tony Gqrcia & Dawn #Men #Diddy
#Melle #Foxxe #STING #Mo #Luke #Big T #Bun B #Die #Tone #Houston #Young D  #Rita
#erson #Evan #Mez #Mary Mary #Big #Rich #Jessica #Teo 

# CASH CASH is off #Bas is off #RBE #Keith

In [99]:
to_drop = [3931,303,4425,4426,5532,4433,4434,7588,7761,8900,8901,8902,8949,9061,9346,9347,9348,9350,9351,9356,9357,9958,
          9960,9961,10882,10883,10994,10996,10999,11011,11352,11354,11714,11716,11721,13066,13067]

to_drop.extend([i for i in range(3908,3917)])
to_drop.extend([i for i in range(3923,3930)])
to_drop.extend([i for i in range(3933,3937)])
to_drop.extend([i for i in range(7593,7597)])
to_drop.extend([i for i in range(8644,8653)])
to_drop.extend([i for i in range(8917,8922)])
to_drop.extend([i for i in range(9362,9376)])
to_drop.extend([i for i in range(10229,10235)])
to_drop.extend([i for i in range(11012,11019)])
to_drop.extend([i for i in range(11342,11348)])
to_drop.extend([i for i in range(11693,11697)])
to_drop.extend([i for i in range(11269,11297)])

for i in to_drop:
    if i in artist_df.index:
        artist_df =artist_df.drop(i)


In [73]:
import numpy as np
keywords = ["Featuring","Feauring", "featuring", "&",","," X ", " x ","Presents", "From", "Or","And",]
new_df = artist_df
temp_list = np.asarray(new_df["artist"])
count=0
for keyword in keywords:
    artist_list = [i.split(keyword) for i in temp_list]
    temp_list = list()
    count=count+1
    for i in artist_list:
            temp_list.extend(i)

artist_list = list()
for i in temp_list:
    i = i.strip()
    if len(i)>1:
        artist_list.append(i)
artist_list = set(artist_list)        
artist_list_new = set(artist_list) 

for i in artist_list:
    if len(artist_df[artist_df["artist"] == i]) >0 and i in artist_list_new:
        artist_list_new.remove(i)
    elif len(artist_df[artist_df["artist"] == i]) >0 and i in artist_list_new:
        artist_list_new.remove(i)

In [ ]:
from tqdm import tqdm_notebook as tqdm
driver = webdriver.Chrome('/Users/iliasmiraoui/Desktop/chromedriver')
pbar = list(artist_list_new)
i = 0
for artist in pbar:
    if (artist not in artist_df["artist"].unique()) and (artist not in artist_df["artist"].unique()):
        i=i+1
        df = get_riaa(artist)
        artist_df = artist_df.append(df)
        if len(df) == 0:
            artist_df = artist_df.append([artist,None,None,None,None,None,None])
        print(i)
        if i % 5 == 0:
            with open('artist_df.pkl', 'wb') as f:
                pickle.dump(artist_df, f)
with open('artist_df.pkl', 'wb') as f:
    pickle.dump(artist_df, f)

In [88]:
for row in artist_df.index:
    if not similar(artist_df.loc[row,"artist"].upper(),artist_df.loc[row,"artists"],40):
        print(fuzz.partial_ratio(artist_df.loc[row,"artist"].upper(),artist_df.loc[row,"artists"]))
        print(row, artist_df.loc[row,"artist"],artist_df.loc[row,"artists"])

37
148 Juice WRLD & YoungBoy Never Broke Again ELLIE GOULDING & JUICE WRLD
38
1434 Lady Gaga & Bradley Cooper TONY BENNETT & LADY GAGA
38
1549 Daddy Yankee & Katy Perry Featuring Snow PLAY-N-SKILLZ FEAT. DADDY YANKEE
0
11101 Die Shordie Shordie


In [107]:
artist_df = artist_df.dropna()

In [108]:
artist_df

,artist,artists,details,format,label,last_certification,songs
0,Lewis Capaldi,LEWIS CAPALDI,"[NOVEMBER 8, 2018, Platinum | August 15, 2019,...",SINGLE\nMORE DETAILS,CAPITOL RECORDS,"August 15, 2019",SOMEONE YOU LOVED
1,Post Malone,POST MALONE & SWAE LEE,"[OCTOBER 18, 2018, 8x Multi-Platinum | October...",SINGLE\nMORE DETAILS,REPUBLIC RECORDS,"October 22, 2019",SUNFLOWER (SPIDER-MAN: INTO THE SPIDER-VERSE)
2,Post Malone,POST MALONE,"[DECEMBER 9, 2016, 10x Multi-Platinum | Octobe...",SINGLE\nMORE DETAILS,REPUBLIC RECORDS,"October 8, 2019",CONGRATULATIONS
3,Post Malone,POST MALONE,"[APRIL 27, 2018, Platinum | March 13, 2019, No...",SINGLE\nMORE DETAILS,REPUBLIC RECORDS,"March 13, 2019",BALL FOR ME
4,Post Malone,POST MALONE,"[APRIL 27, 2018, 3x Multi-Platinum | March 13,...",ALBUM\nMORE DETAILS,REPUBLIC RECORDS,"March 13, 2019",BEERBONGS & BENTLEYS
...,...,...,...,...,...,...,...
13338,Yandel,WISIN Y YANDEL,"[MAY 5, 2012, Gold | May 9, 2013, DUO, Digital...",SINGLE\nMORE DETAILS,MACHETE MUSIC,"May 9, 2013",FOLLOW THE LEADER
13339,Yandel,WISIN Y YANDEL PRESENTS DJ NESTY,"[MARCH 13, 2008, Platinum | March 27, 2009, DU...",ALBUM\nMORE DETAILS,MACHETE MUSIC,"March 27, 2009",WISIN Y YANDEL PRESENTA: LA MENTE MAESTRA
13340,Yandel,WISIN Y YANDEL,"[NOVEMBER 6, 2007, 3x Multi-Platinum | April 1...",ALBUM\nMORE DETAILS,MACHETE MUSIC,"April 11, 2008",LOS EXTRATERRESTRES
13341,Yandel,WISIN Y YANDEL,"[NOVEMBER 8, 2005, Gold | September 12, 2006, ...",ALBUM\nMORE DETAILS,MACHETE MUSIC,"September 12, 2006",PA'L MUNDO
